# Presence only data modelling
Build presence only models using presence and background data for training; and absence and pseudo-absence for testing. 

## Downloads and imports

In [ ]:
# Install if you are are't running through docker
# install.packages(c("dismo","maxnet","tictoc","MLmetrics","dplyr"),repos = "http://cran.us.r-project.org")

In [3]:
library(maxnet)       
library(tictoc)          
library(MLmetrics)
library(dplyr)


Attaching package: ‘MLmetrics’


The following object is masked from ‘package:base’:

    Recall



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




## Helper functions

In [4]:
replace_na_with_mean <- function(dataframe) {
    # Replace NA in all columns with mean of column
    for(i in 1:ncol(dataframe)) {                                   
        dataframe[ , i][is.na(dataframe[ , i])] <- mean(dataframe[ , i], na.rm = TRUE)
    }
    return(dataframe)
}

In [5]:
train_test_evaluate <- function(training_data,test_data,classes,regmult,plot_diagram) {
    # Generate and normalize train data
    train_x <- training_data[, -which(names(training_data) == "presence")]


    ## Remove nans
    train_x = replace_na_with_mean(train_x) 
    
    ## Train model
    maxent_model_for_run <- maxnet(training_data$presence, train_x, maxnet.formula(training_data$presence, train_x, classes=classes),regmult=regmult)
    
    # Generate and normalize test data
    ## Ignore real absences for now - we eval on pseudo absences - ignore presence=2
    test_data = test_data[test_data$presence %in% c("0", "1"), ]

    test_x <- test_data[, -which(names(test_data) == "presence")]

    # Remove nans
    test_x = replace_na_with_mean(test_x) 

    # Test model
    test_pred = predict(maxent_model_for_run, test_x, clamp=T, type="cloglog")
    auc <- AUC(y_pred=test_pred, y_true=test_data$presence)
    accuracy <- Accuracy(y_pred=ifelse(test_pred >= .5, 1, 0), y_true=test_data$presence)
    f1 <- F1_Score(y_pred=ifelse(test_pred >= .5, 1, 0), y_true=test_data$presence)
    result_test <- data.frame(accuracy,f1,auc)

    return(result_test)
}

train_test_evaluate_existing_model <- function(maxent_model,test_data,plot_diagram) {
    tic()
    # Generate and normalize test data
    ## Ignore real absences for now - we eval on pseudo absences - ignore presence=2
    test_data = test_data[test_data$presence %in% c("0", "1"), ]

    test_x <- test_data[, -which(names(test_data) == "presence")]

    # Remove nans
    test_x = replace_na_with_mean(test_x) 

    # Test model
    test_pred = predict(maxent_model, test_x, clamp=T, type="cloglog")
    auc <- AUC(y_pred=test_pred, y_true=test_data$presence)
    accuracy <- Accuracy(y_pred=ifelse(test_pred >= .5, 1, 0), y_true=test_data$presence)
    f1 <- F1_Score(y_pred=ifelse(test_pred >= .5, 1, 0), y_true=test_data$presence)
    result_test <- data.frame(accuracy,f1,auc)
    toc()
    return(result_test)
}

In [6]:
# randomly sample from dataframe
sample_from_dataset <- function(dataset,sample_percentage) {
    num_training_samples = nrow(dataset)
    smp_size <- floor(sample_percentage * num_training_samples)
    train_ind <- sample(seq_len(num_training_samples), size = smp_size)
    return(dataset[train_ind, ])
}

# Build models

## Loading data

In [7]:
# Load preprocessed and scaled data
train <- read.csv("../data/presence_only/scaled_train.csv", header=TRUE)
validation <- read.csv("../data/presence_only/scaled_val.csv", header=TRUE)


In [8]:
sprintf("Validation is %s of train set. Presence points is %s, psuedo-absense/background is %s ", nrow(validation)/(nrow(train)+nrow(validation)),nrow(validation[validation$presence=="1",])/nrow(validation), nrow(validation[validation$presence=="0",])/nrow(validation))

[1] "Validation is 0.200018520233355 of train set. Presence points is 0.500462962962963, psuedo-absense/background is 0.499537037037037 "

## Hyperparam Search (**Optional**)
Use train and validation to find best hyparams

#### Use the train and validation set to find good hyperparams (or a subset of train and validation if this takes too long)

In [ ]:
# Only use some of the training data - this takes too long while all the training data
train_sample = sample_from_dataset(train,0.25)
validation_sample = validation

print(nrow(train_sample))
print(ncol(train_sample))

# Create search grid
search_grid <- expand.grid(classes=c("l","lq","lqh","h","t","p"),regmult=seq(0.25, 2, by=0.25))
nrow(search_grid)

best_auc = 0
# Loop through search grid and find best model and params
for(row in 1:nrow(search_grid)){
  tic()
    
  # Get vars
  regmult <- search_grid$regmult[row]
  classes <- search_grid$classes[row]
    
  cat(sprintf("Running config %s out of %s - reg mult:%s classes:%s \n", row,nrow(search_grid), regmult,classes))
  result = train_test_evaluate(train_sample,validation_sample,classes,regmult,plot_diagram=FALSE)
  print(result)
  if(best_auc < result$auc){
   best_result <- result
   best_class <- classes
   best_regmult <- regmult
   best_auc <-  result$auc
   print("New best model")
   print(best_result)
  }
  
  flush.console()
  toc()
}

#### Use best params found on train & val sets, and use those on the test set

In [ ]:
print(best_class)
print(best_regmult)

# Run Best Configuration on Test Data slices

Use these params if you are not running hyperparam tuning.

In [9]:
best_class <- "l"
best_regmult <- 1 

In [10]:
tic()
training_data = rbind(validation, train)

training_data_subset_cols <- training_data

all_results <- data.frame(index=integer(),accuracy=double(),f1=double(),
                 auc=double()) 

train_x <- training_data_subset_cols[, -which(names(training_data_subset_cols) == "presence")]

## Remove nans
train_x = replace_na_with_mean(train_x) 

## Train model
maxent_model <- maxnet(training_data_subset_cols$presence, train_x, maxnet.formula(training_data_subset_cols$presence, train_x, classes=best_class),regmult=best_regmult)
toc()

for (i in seq(7, 700, by=7)){
    tic()
    test_filename <- sprintf("../data/test_data/test_scaled_seed_%s.csv", i)
    
    test_data <- read.csv(test_filename, header=TRUE)
    
    test_data <- test_data 
    
    test_result <- train_test_evaluate_existing_model(maxent_model,test_data,plot_diagram=FALSE)
    test_result[, "seed"] <- i
    all_results = rbind(all_results, test_result)
    
    flush.console()
    toc()
}

print(all_results)
results_filename <- sprintf("../results/presence-only/maxent_%s_class_%s_reg.csv", best_class,best_regmult)
write.csv(all_results,results_filename,row.names=FALSE)

2387.335 sec elapsed
0.092 sec elapsed
0.836 sec elapsed
0.401 sec elapsed
1.579 sec elapsed
0.092 sec elapsed
0.794 sec elapsed
0.091 sec elapsed
1.13 sec elapsed
0.665 sec elapsed
1.58 sec elapsed
0.092 sec elapsed
0.818 sec elapsed
0.695 sec elapsed
1.616 sec elapsed
0.102 sec elapsed
0.775 sec elapsed
0.726 sec elapsed
1.628 sec elapsed
0.094 sec elapsed
0.761 sec elapsed
0.396 sec elapsed
1.651 sec elapsed
0.093 sec elapsed
0.841 sec elapsed
0.414 sec elapsed
1.707 sec elapsed
0.086 sec elapsed
0.79 sec elapsed
0.448 sec elapsed
1.321 sec elapsed
0.09 sec elapsed
0.766 sec elapsed
0.656 sec elapsed
1.476 sec elapsed
0.087 sec elapsed
0.764 sec elapsed
0.711 sec elapsed
1.573 sec elapsed
0.09 sec elapsed
0.75 sec elapsed
0.71 sec elapsed
1.563 sec elapsed
0.091 sec elapsed
0.719 sec elapsed
0.384 sec elapsed
1.717 sec elapsed
0.088 sec elapsed
0.722 sec elapsed
0.709 sec elapsed
1.568 sec elapsed
0.087 sec elapsed
0.702 sec elapsed
0.445 sec elapsed
1.319 sec elapsed
0.086 sec elap